In [92]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from pathlib import *
from pymongo import *

from exp.nb_00 import *
from exp.nb_01 import *
from exp.nb_02 import *

In [56]:
WORKPLACE = Path('/chia_cluster/home/jackyu/radiation/analysis_-_new_kdist_param_sw/')

In [75]:
PARAM = LBLnewBestfitSWParam(band=9, vmin=8200, vmax=14290, nv=10000, dv=0.001,
                     molecule='h2o', conc='atmpro',
                     ref_pts=[(30, 250), (300, 250)], ng_refs=[4, 8], ng_adju=[-3, 0],
                     wgt=[(.95, .9, .6, .55), (.55, .55, .55, .6, .7, .75, .85, .95)],
                     cosz=1., rsfc=0,
                     klin=3e-25,
                     option_k_lookup=0,
                     atmpro='saw',
                     commitnumber='79f2292')

In [76]:
PATH = Path('/'.join(sorted([f'{n}_{v}' for n, v in vars(PARAM).items()])))
PATH
#PATH = 'test1'

PosixPath('atmpro_saw/band_9/commitnumber_79f2292/conc_atmpro/cosz_1.0/dv_0.001/klin_3e-25/molecule_h2o/ng_adju_[-3, 0]/ng_refs_[4, 8]/nv_10000/option_k_lookup_0/ref_pts_[(30, 250), (300, 250)]/rsfc_0/tsfc_None/vmax_14290/vmin_8200/wgt_[(0.95, 0.9, 0.6, 0.55), (0.55, 0.55, 0.55, 0.6, 0.7, 0.75, 0.85, 0.95)]')

## Run lblnew-bestfit-sw  (the fortran code)

In [77]:
runner = LBLnewBestfitSWRun(path=PATH, param=PARAM)

In [78]:
runner.param

<class 'exp.nb_00.LBLnewBestfitSWParam'>
{'dv': 0.001, 'nv': 10000, 'commitnumber': '79f2292', 'band': 9, 'molecule': 'h2o', 'atmpro': 'saw', 'tsfc': None, 'vmin': 8200, 'vmax': 14290, 'conc': 'atmpro', 'ref_pts': [(30, 250), (300, 250)], 'ng_refs': [4, 8], 'ng_adju': [-3, 0], 'wgt': [(0.95, 0.9, 0.6, 0.55), (0.55, 0.55, 0.55, 0.6, 0.7, 0.75, 0.85, 0.95)], 'cosz': 1.0, 'rsfc': 0, 'klin': 3e-25, 'option_k_lookup': 0}

In [79]:
list(runner.path.iterdir())

[PosixPath('atmpro_saw/band_9/commitnumber_79f2292/conc_atmpro/cosz_1.0/dv_0.001/klin_3e-25/molecule_h2o/ng_adju_[-3, 0]/ng_refs_[4, 8]/nv_10000/option_k_lookup_0/ref_pts_[(30, 250), (300, 250)]/rsfc_0/tsfc_None/vmax_14290/vmin_8200/wgt_[(0.95, 0.9, 0.6, 0.55), (0.55, 0.55, 0.55, 0.6, 0.7, 0.75, 0.85, 0.95)]/lblnew-bestfit-sw.f'),
 PosixPath('atmpro_saw/band_9/commitnumber_79f2292/conc_atmpro/cosz_1.0/dv_0.001/klin_3e-25/molecule_h2o/ng_adju_[-3, 0]/ng_refs_[4, 8]/nv_10000/option_k_lookup_0/ref_pts_[(30, 250), (300, 250)]/rsfc_0/tsfc_None/vmax_14290/vmin_8200/wgt_[(0.95, 0.9, 0.6, 0.55), (0.55, 0.55, 0.55, 0.6, 0.7, 0.75, 0.85, 0.95)]/lblcom.f')]

In [84]:
os.chdir('sw_dev')

In [85]:
! pwd

/chia_cluster/home/jackyu/climatools/climatools/notebooks/sw_dev


In [86]:
runner.input_params()

In [87]:
proc = runner.build()
out, err = proc.communicate()
print(out.decode())
print(err.decode())


lblcom.f(169): remark #8290: Recommended relationship between field width 'W' and the number of fractional digits 'D' in this edit descriptor is 'W>=D+3'.
 900  format(f12.6,e10.3,f5.4,f10.4,a53,i3,i4,i3)
--------------------------^
lblnew-bestfit-sw.f(244): remark #7712: This variable has not been used.   [D_FLUX]
      real d_flux(nlayer+1,ng),d_tflux(nlayer+1) 
-----------^
lblnew-bestfit-sw.f(244): remark #7712: This variable has not been used.   [D_TFLUX]
      real d_flux(nlayer+1,ng),d_tflux(nlayer+1) 
-------------------------------^
lblnew-bestfit-sw.f(281): remark #7712: This variable has not been used.   [ABSCOMZE]
      real*8 accu6th, abscomze 
----------------------^
lblnew-bestfit-sw.f(298): remark #7712: This variable has not been used.   [P_KTABLE]
      real :: p_ktable(nl,nt), t_ktable(nl,nt), xk
--------------^
lblnew-bestfit-sw.f(298): remark #7712: This variable has not been used.   [T_KTABLE]
      real :: p_ktable(nl,nt), t_ktable(nl,nt), xk
-------------------

In [88]:
list(runner.path.iterdir())

[PosixPath('atmpro_saw/band_9/commitnumber_79f2292/conc_atmpro/cosz_1.0/dv_0.001/klin_3e-25/molecule_h2o/ng_adju_[-3, 0]/ng_refs_[4, 8]/nv_10000/option_k_lookup_0/ref_pts_[(30, 250), (300, 250)]/rsfc_0/tsfc_None/vmax_14290/vmin_8200/wgt_[(0.95, 0.9, 0.6, 0.55), (0.55, 0.55, 0.55, 0.6, 0.7, 0.75, 0.85, 0.95)]/lblnew-bestfit-sw.f'),
 PosixPath('atmpro_saw/band_9/commitnumber_79f2292/conc_atmpro/cosz_1.0/dv_0.001/klin_3e-25/molecule_h2o/ng_adju_[-3, 0]/ng_refs_[4, 8]/nv_10000/option_k_lookup_0/ref_pts_[(30, 250), (300, 250)]/rsfc_0/tsfc_None/vmax_14290/vmin_8200/wgt_[(0.95, 0.9, 0.6, 0.55), (0.55, 0.55, 0.55, 0.6, 0.7, 0.75, 0.85, 0.95)]/lblnew-bestfit-sw.exe'),
 PosixPath('atmpro_saw/band_9/commitnumber_79f2292/conc_atmpro/cosz_1.0/dv_0.001/klin_3e-25/molecule_h2o/ng_adju_[-3, 0]/ng_refs_[4, 8]/nv_10000/option_k_lookup_0/ref_pts_[(30, 250), (300, 250)]/rsfc_0/tsfc_None/vmax_14290/vmin_8200/wgt_[(0.95, 0.9, 0.6, 0.55), (0.55, 0.55, 0.55, 0.6, 0.7, 0.75, 0.85, 0.95)]/lblcom.f')]

In [89]:
proc = runner.run()
out, err = proc.communicate()
print(out.decode())
print(err.decode())

Job <433536> is submitted to queue <serial>.




In [91]:
! qstat -u jackyu

serial;  type=BATCH;  [ENABLED];  pri=35
4 run;   9 wait;

        REQUEST NAME        REQUEST ID          USER   STATE
   1:   fitsw_9_h2o_            433535       jackyu  RUNNING
   2:   fitsw_9_h2o_            433536       jackyu  RUNNING

v4q;  type=BATCH;  [ENABLED];  pri=30
288 run;   0 wait;


v4qexp;  type=BATCH;  [ENABLED];  pri=30
144 run;   0 wait;


LCCR_Q;  type=BATCH;  [ENABLED];  pri=30
16 run;   0 wait;


URBAN_Q;  type=BATCH;  [ENABLED];  pri=30
1121 run;   448 wait;


orc;  type=BATCH;  [ENABLED];  pri=30
0 run;   0 wait;


RCEC_Q;  type=BATCH;  [ENABLED];  pri=30
192 run;   0 wait;


medium_priority;  type=BATCH;  [ENABLED];  pri=10
0 run;   0 wait;




In [ ]:
runner.save2mongodb(client)